In [1]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score

In [5]:
df = pd.read_csv('labeled_data.csv')

In [7]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [8]:
def clean_tweet(tweet):
    tweet = re.sub(r'http\S+', '', tweet)
    tweet = re.sub(r'@\w+', '', tweet)
    tweet = re.sub(r'#\w+', '', tweet)
    tweet = re.sub(r'\d+', '', tweet)
    tweet = tweet.lower()
    tweet = re.sub(r'\W', ' ', tweet)
    tweet = re.sub(r'\s+', ' ', tweet)
    return tweet

In [9]:
df['clean_tweet'] = df['tweet'].apply(clean_tweet)

In [10]:
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [11]:
def preprocess_tweet(tweet):
    words = tweet.split()
    words = [lemmatizer.lemmatize(word) for word in words if word not in stop_words]
    return ' '.join(words)

In [12]:
df['processed_tweet'] = df['clean_tweet'].apply(preprocess_tweet)

In [13]:
max_words = 10000
max_len = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(df['processed_tweet'])
sequences = tokenizer.texts_to_sequences(df['processed_tweet'])
padded_sequences = pad_sequences(sequences, maxlen=max_len)

In [14]:
X = padded_sequences
y = df['class']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [16]:
model = Sequential()
model.add(Embedding(input_dim=max_words, output_dim=128, input_length=max_len))
model.add(LSTM(128, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(3, activation='softmax'))

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


In [19]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
early_stopping = EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True, verbose=1)

In [ ]:
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_split=0.2, callbacks=[early_stopping])

Epoch 1/20
496/496 ━━━━━━━━━━━━━━━━━━━━ 91s 179ms/step - accuracy: 0.8244 - loss: 0.5133 - val_accuracy: 0.9080 - val_loss: 0.2525
Epoch 2/20
496/496 ━━━━━━━━━━━━━━━━━━━━ 142s 179ms/step - accuracy: 0.9220 - loss: 0.2225 - val_accuracy: 0.9004 - val_loss: 0.2739
Epoch 3/20
496/496 ━━━━━━━━━━━━━━━━━━━━ 88s 177ms/step - accuracy: 0.9465 - loss: 0.1606 - val_accuracy: 0.9062 - val_loss: 0.2883
Epoch 4/20
496/496 ━━━━━━━━━━━━━━━━━━━━ 96s 194ms/step - accuracy: 0.9565 - loss: 0.1217 - val_accuracy: 0.9002 - val_loss: 0.3390
Epoch 5/20
496/496 ━━━━━━━━━━━━━━━━━━━━ 135s 179ms/step - accuracy: 0.9668 - loss: 0.0887 - val_accuracy: 0.8949 - val_loss: 0.4191
Epoch 6/20
496/496 ━━━━━━━━━━━━━━━━━━━━ 91s 183ms/step - accuracy: 0.9738 - loss: 0.0715 - val_accuracy: 0.8850 - val_loss: 0.4387
Epoch 6: early stopping
Restoring model weights from the end of the best epoch: 1.


In [ ]:
y_pred_proba = model.predict(X_test)
y_pred = np.argmax(y_pred_proba, axis=1)
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred, average='weighted')
roc_auc = roc_auc_score(y_test, y_pred_proba, multi_class='ovo', average='weighted')

155/155 ━━━━━━━━━━━━━━━━━━━━ 7s 45ms/step


In [ ]:
print(f"Accuracy: {accuracy:.4f}")
print(f"F1-score (weighted): {f1:.4f}")
print(f"ROC AUC (weighted, ovo): {roc_auc:.4f}")

Accuracy: 0.8997
F1-score (weighted): 0.8865
ROC AUC (weighted, ovo): 0.9182


In [ ]:
def test_input(text):
    cleaned_text = clean_tweet(text)
    processed_text = preprocess_tweet(cleaned_text)
    sequence = tokenizer.texts_to_sequences([processed_text])
    padded_sequence = pad_sequences(sequence, maxlen=max_len)
    predicted_probabilities = model.predict(padded_sequence)[0]
    predicted_class = np.argmax(predicted_probabilities)
    class_labels = {0: 'Hate Speech', 1: 'Offensive Language', 2: 'Neither'}
    print(f"Input: {text}")
    print(f"Predicted Class: {class_labels[predicted_class]}")
    print("Class Probabilities:")
    for cls, prob in zip(class_labels.values(), predicted_probabilities):
        print(f"  {cls}: {prob:.4f}")

In [ ]:
example_text = "hello"
test_input(example_text)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
Input: hello
Predicted Class: Neither
Class Probabilities:
  Hate Speech: 0.1481
  Offensive Language: 0.4093
  Neither: 0.4426


In [ ]:
model.save('hate_speech_model.h5')

In [ ]:
import pickle
with open('tokenizer.pkl', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)